<a href="https://colab.research.google.com/github/architectyou/LangChain/blob/main/LangChain_docs/agent_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install -q langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install -q langchainhub

In [ ]:
import getpass
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [ ]:
from langchain_core.tools import tool

@tool
def multiply(first_int : int, second_int : int)->int:
  """Multiply two integers together."""
  return first_int * second_int

In [ ]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [ ]:
multiply.invoke({"first_int":4, "second_int":5})

20

### tool / function calling

calling api = function calling
tool calling을 지원하는 모델들이 정해져있음. 확인해봐야 함

https://python.langchain.com/v0.1/docs/integrations/chat/

chat openai는 비동기 invoke, 비동기 streaming, (streaming)을 지원하지 않음 -> 참고할것

In [ ]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-3.5-turbo-0125")

In [ ]:
# bind_tool 사용
llm_with_tools = llm.bind_tools([multiply])

In [ ]:
msg = llm_with_tools.invoke("whats 5 times forty two")
msg.tool_calls

[{'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 42},
  'id': 'call_Eo2qJ5m7EHvtLh8H8VMTFVjF'}]

invoking tools -> tool 호출하기

In [ ]:
from operator import itemgetter

chain = llm_with_tools | (lambda x : x.tool_calls[0]["args"]) | multiply
chain.invoke("What's four times 23?")

92

모델이 tool을 몇개를 사용할 지 고르고 싶게 하고 싶을 때!

- agent를 이용하면 모델이 어떤 순서에 툴을 이용할지를 스스로 결정하게 된다.
- tool calling agent를 이용하면 가장 관련있는 tool을 알아서 불러와준다.

- agent type들 확인하기
https://python.langchain.com/v0.1/docs/modules/agents/agent_types/

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [ ]:
# langchain-hub 에서 prompt 가져오기

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [ ]:
# Chains with multiple tools

@tool
def add(first_int : int, second_int : int) -> int:
  "Add two integers."
  return first_int + second_int

@tool
def exponentiate(base : int, exponent : int) -> int:
  "Exponentiate the base to the exponent power."
  return base**exponent

tools = [multiply, add, exponentiate]

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
# agent executor 동작
agent_executor = AgentExecutor(agent=agent, tools = tools, verbose = True)

In [ ]:
agent_executor.invoke(
    {
        "input" : "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.'}

원하는 format으로 받고 싶기 때문에 prompt 를 새로 만들어준다.

(arguments of tools)
- json blob
- render : ~하게 하다, 되게 하다

In [ ]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description([multiply])
rendered_tools

'multiply(first_int: int, second_int: int) -> int - Multiply two integers together.'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool :

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments'
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [ ]:
prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool : \n\nmultiply(first_int: int, second_int: int) -> int - Multiply two integers together.\n\nGiven the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments'\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [ ]:
# output parser

from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model = "gpt-3.5-turbo", temperature = 0)
chain = prompt | model | JsonOutputParser()
chain.invoke({"input" : "What's thirteen times 4"})

{'name': 'multiply', 'arguments': {'first_int': 13, 'second_int': 4}}

In [ ]:
# invoking the tool

from operator import itemgetter
# tool의 arguments들을 가져와서 arguments들을 이용하고 싶을 때
# json output parser를 이용해서 값을 parsing 하고 그 값을 통해 연산을 수행하고 싶을 때
chain = prompt | model | JsonOutputParser() | itemgetter("arguments") | multiply
chain.invoke({"input" : "what's thirteen times 4"})

52

### Mutliple tools로 부터 고르게 하고 싶을 때

In [ ]:
def tool_chain(model_output):
  tool_map = {tool.name : tool for tool in tools}
  chosen_tool = tool_map[model_output["name"]]
  return itemgetter("arguments") | chosen_tool

In [ ]:
rendered_tools = render_text_description(tools)
system_prompt = f"""
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool :

{rendered_tools}

Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system" , system_prompt), ("user", "{input}")]
)

chain = prompt | model | JsonOutputParser() | tool_chain
chain.invoke({"input" : "what's 3 plus 1132"})

1135

RunnablePassthrough -> 이용해서 arguments 확인하기

어떤 모델을 사용했는지도 출력해줌

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output = tool_chain)
)
chain.invoke({"input" : "What's 3 plus 1132"})

{'name': 'add',
 'arguments': {'first_int': 3, 'second_int': 1132},
 'output': 1135}

In [ ]:
chain.invoke({"input" : "what's 5 times 4"})

{'name': 'multiply',
 'arguments': {'first_int': 5, 'second_int': 4},
 'output': 20}

### Function calling 중간에 인간이 개입하기

In [ ]:
from operator import itemgetter
from typing import Dict, List

from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import tool

@tool
def count_emails(last_n_days : int) -> int:
  """Multiply two integers together."""
  return last_n_days * 2

@tool
def send_email(message : str, recipient : str) -> str:
  "Add two integers."
  return f"Successfully sent email to {recipient}."

tools = [count_emails, send_email]
llm_with_tools = llm.bind_tools(tools)

def call_tools(msg : AIMessage) -> List[Dict]:
  """Simple sequential tool calling helper."""
  tool_map = {tool.name : tool for tool in tools}
  tool_calls = msg.tool_calls.copy()
  for tool_call in tool_calls:
    tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
  return tool_calls

chain = llm_with_tools | call_tools
chain.invoke("how many emails did i get in the last 5 days?")

[{'name': 'count_emails',
  'args': {'last_n_days': 5},
  'id': 'call_SOyl3MEE9im8ZFxZ7zJln4S3',
  'output': 10}]

In [ ]:
import json

def human_approval(msg : AIMessage) -> Runnable :
  tool_strs = "\n\n".join(
      json.dumps(tool_call, indent = 2) for tool_call in msg.tool_calls
  )
  input_msg = (
      f"Do you approve of the following tool invocations\n\n{tool_strs}\n\n"
      "Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no."
  )
  resp = input(input_msg)
  if resp.lower() not in ("yes", "y"):
    raise ValueError(f"Tool invocations not approved : \n\n {tool_strs}")
  return msg

In [ ]:
chain = llm_with_tools | human_approval | call_tools
chain.invoke("how many emails did i get in the last 5 days?")

Do you approve of the following tool invocations

{
  "name": "count_emails",
  "args": {
    "last_n_days": 5
  },
  "id": "call_uQkbMuIR4qzLnxnXjjTdFzFE"
}

Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no.y


[{'name': 'count_emails',
  'args': {'last_n_days': 5},
  'id': 'call_uQkbMuIR4qzLnxnXjjTdFzFE',
  'output': 10}]

In [ ]:
chain.invoke("Send sally@gmail.com an email saying 'What's up homie'")

Do you approve of the following tool invocations

{
  "name": "send_email",
  "args": {
    "message": "What's up homie",
    "recipient": "sally@gmail.com"
  },
  "id": "call_3MzZmKgtW5oJ9k73njHHcEEU"
}

Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no.y


[{'name': 'send_email',
  'args': {'message': "What's up homie", 'recipient': 'sally@gmail.com'},
  'id': 'call_3MzZmKgtW5oJ9k73njHHcEEU',
  'output': 'Successfully sent email to sally@gmail.com.'}]